# Account


In [ ]:
# | default_exp classes.DomoAccount

In [ ]:
# | export

from enum import Enum
from dataclasses import dataclass, field
from abc import ABC, abstractmethod

from typing import Union

import datetime as dt
import re


import httpx

from fastcore.basics import patch_to

import domolibrary.utils.convert as cd
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.account as account_routes

# Account Connector Config

Each Domo Dataset that pulls data into Vault must have a stream, which stores the configuration information related to which data is exctracted from a datasource. Each stream has an associated account which stores the source data's authentication information.

Because each datasource may have different authentication parameters, there may be multiple versions of the same account type (with different credentials) or multiple account types deployed within a domo instance if the user is extracting data from multiple systems.

Account's can be configured such that certain fields are designated as encrypted fields, and the user will never be able to see the encrypted values EXCEPT in Domo's Jupyter Notebook integration.


In [ ]:
# | export
class DomoAccount_Config(ABC):
    """DomoAccount Config abstract base class"""

    data_provider_type: str

    @classmethod
    @abstractmethod
    def _from_json(cls, obj):
        """convert accounts API response into a class object"""
        pass

    @abstractmethod
    def to_json(self):
        """convert class object into a format the accounts API expects"""
        pass


In [ ]:
# | export
@dataclass
class DomoAccount_Config_AbstractCredential(DomoAccount_Config):
    data_provider_type = "abstract-credential-store"
    credentials: dict

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            credentials=dd.credentials,
        )

    def to_json(self):
        return {"credentials": self.credentials}


In [ ]:
# | export
@dataclass
class DomoAccount_Config_DatasetCopy(DomoAccount_Config):
    domo_instance: str
    access_token: str = field(repr=False)

    data_provider_type = "dataset-copy"

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(access_token=dd.accessToken, domo_instance=dd.instance)

    def to_json(self):
        return {"accessToken": self.access_token, "instance": self.domo_instance}


In [ ]:
# | export
@dataclass
class DomoAccount_Config_Governance(DomoAccount_Config):
    domo_instance: str
    access_token: str = field(repr=False)

    data_provider_type = "domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600"

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(access_token=dd.apikey, domo_instance=dd.customer)

    def to_json(self):
        return {"apikey": self.access_token, "customer": self.domo_instance}


In [ ]:
# | export
@dataclass
class DomoAccount_Config_HighBandwidthConnector(DomoAccount_Config):
    aws_access_key: str
    aws_secret_key: str = field(repr=False)
    s3_staging_dir: str

    region: str = "us-west-2"
    data_provider_type = "amazon-athena-high-bandwidth"

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            aws_access_key=dd.awsAccessKey,
            aws_secret_key=dd.awsSecretKey,
            s3_staging_dir=dd.s3StagingDir,
            region=dd.region,
        )

    def to_json(self):
        return {
            "awsAccessKey": self.aws_access_key,
            "awsSecretKey": self.aws_secret_key,
            "s3StagingDir": self.s3_staging_dir,
            "region": self.region,
        }


In [ ]:
# | export
@dataclass
class DomoAccount_Config_AwsAthena(DomoAccount_Config):
    aws_access_key: str
    aws_secret_key: str = field(repr=False)
    s3_staging_dir: str
    workgroup: str

    region: str = "us-west-2"
    data_provider_type = "aws-athena"

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            aws_access_key=dd.awsAccessKey,
            aws_secret_key=dd.awsSecretKey,
            s3_staging_dir=dd.s3StagingDir,
            region=dd.region,
            workgroup=dd.workgroup,
        )

    def to_json(self):
        return {
            "awsAccessKey": self.aws_access_key,
            "awsSecretKey": self.aws_secret_key,
            "s3StagingDir": self.s3_staging_dir,
            "region": self.region,
            "workgroup": self.workgroup,
        }

In [ ]:
# | export
class AccountConfig(Enum):
    """
    Enum provides appropriate spelling for data_provider_type and config object.
    The name of the enum should correspond with the data_provider_type with hyphens replaced with underscores.
    """

    amazon_athena_high_bandwidth = DomoAccount_Config_HighBandwidthConnector

    abstract_credential_store = DomoAccount_Config_AbstractCredential

    dataset_copy = DomoAccount_Config_DatasetCopy

    domo_governance_d14c2fef_49a8_4898_8ddd_f64998005600 = DomoAccount_Config_Governance

    aws_athena = DomoAccount_Config_AwsAthena

# MAIN -- DomoAccount


In [ ]:
# | export
@dataclass
class DomoAccount:
    name: str
    data_provider_type: str

    id: int = None
    created_dt: dt.datetime = None
    modified_dt: dt.datetime = None
    auth: dmda.DomoAuth = field(repr=False, default=None)

    config: DomoAccount_Config = None

    @classmethod
    def _from_json(cls, obj: dict, auth: dmda.DomoAuth = None):
        """converts data_v1_accounts API response into an accounts class object"""

        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.id,
            name=dd.displayName,
            data_provider_type=dd.dataProviderType,
            created_dt=cd.convert_epoch_millisecond_to_datetime(dd.createdAt),
            modified_dt=cd.convert_epoch_millisecond_to_datetime(dd.modifiedAt),
            auth=auth,
        )

In [ ]:
# | export
class DomoAccount_DataProviderType_ConfigNotDefined(de.DomoError):
    def __init__(
        self, account_id, data_provider_type, domo_instance, function_name="_get_config"
    ):

        message = f"🛑 data provider type {data_provider_type} for account_id {account_id} not defined yet.  Extend the AccountConfig class"

        super().__init__(
            message=message, function_name=function_name, domo_instance=domo_instance
        )


@patch_to(DomoAccount)
async def _get_config(
    self: DomoAccount, session=None, debug_api: bool = None, return_raw: bool = False
):

    res_config = await account_routes.get_account_config(
        auth=self.auth,
        account_id=self.id,
        data_provider_type=self.data_provider_type,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res_config

    enum_clean = re.sub("-", "_", self.data_provider_type)

    if not enum_clean in AccountConfig.__members__:
        raise DomoAccount_DataProviderType_ConfigNotDefined(
            account_id=self.id,
            data_provider_type=self.data_provider_type,
            domo_instance=self.auth.domo_instance,
        )

    self.config = AccountConfig[enum_clean].value._from_json(res_config.response)

    return self.config

In [ ]:
# | export
@patch_to(DomoAccount, cls_method=True)
async def get_from_id(
    cls,
    auth: dmda.DomoAuth,
    account_id: int,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_api: bool = False,
):

    res = await account_routes.get_account_from_id(
        auth=auth, account_id=account_id, session=session, debug_api=debug_api
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    obj = res.response
    acc = cls._from_json(obj, auth)

    try:
        await acc._get_config(session=session, debug_api=debug_api)

    except DomoAccount_DataProviderType_ConfigNotDefined as e:
        print(e)

    finally:
        return acc

#### sample implementation of get_from_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    await DomoAccount.get_from_id(auth=token_auth, account_id=35, return_raw=False)
except account_routes.GetAccount_NoMatch as e:
    print(e)

get_account_from_id: Status 403 - account_id 35 not found at domo-dojo


## Account Metadata and Configuration


In [ ]:
# | export
@patch_to(DomoAccount)
async def update_config(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    config: DomoAccount_Config = None,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):

    auth = auth or self.auth

    config = config or self.config

    res = await account_routes.update_account_config(
        auth=auth,
        account_id=self.id,
        data_provider_type=self.data_provider_type,
        config_body=config.to_json(),
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    await self._get_config(session=session, debug_api=debug_api)

    return self


#### Sample implementation of update_config


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

domo_instance = "domo-dojo"
access_token = os.environ["DOMO_DOJO_ACCESS_TOKEN"]

# creates a DomoAccount object
domo_account = await DomoAccount.get_from_id(auth=token_auth, account_id=5)


# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.config.domo_instance = "domo-dojo"
domo_account.config.access_token = os.environ["DOMO_DOJO_ACCESS_TOKEN"]
await domo_account.update_config()

# update domo Account API by passing new config object
config = AccountConfig.domo_governance_d14c2fef_49a8_4898_8ddd_f64998005600.value(
    domo_instance=domo_instance, access_token=access_token
)
await domo_account.update_config(config=config)


DomoAccount(name='test_rename', data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 3, 3, 3, 6, 8), config=DomoAccount_Config_Governance(domo_instance='domo-dojo'))

In [ ]:
# | export
class DomoAccount_UpdateName_Error(de.DomoError):
    def __init__(
        self,
        domo_instance,
        status,
        message,
        entity_id,
        function_name="update_name",
    ):

        super().__init__(
            function_name=function_name,
            entity_id=entity_id,
            domo_instance=domo_instance,
            status=status,
            message=message,
        )


@patch_to(DomoAccount)
async def update_name(
    self: DomoAccount,
    account_name: str = None,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):

    auth = auth or self.auth

    # print(auth, self.id, self.data_provider_type, self.config.to_json())

    res = await account_routes.update_account_name(
        auth=auth,
        account_id=self.id,
        account_name=account_name or self.name,
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success:
        raise DomoAccount_UpdateName_Error(
            entity_id=self.id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    self = await self.get_from_id(auth=auth, account_id=self.id)

    return self

#### sample implementation of update_name


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_from_id(auth=token_auth, account_id=5)

account_name = "test_rename"

# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.name = account_name
# await domo_account.update_name()

# update domo Account API by passing account_name str
await domo_account.update_name(account_name=account_name, return_raw=False)


DomoAccount(name='test_rename', data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 3, 3, 3, 38, 45), config=DomoAccount_Config_Governance(domo_instance='domo-dojo'))

## Create Account


In [ ]:
# | export
class DomoAccount_CreateAccount_Error(de.DomoError):
    def __init__(
        self,
        entity_id,
        domo_instance,
        status,
        message,
        function_name="create_account",
    ):

        super().__init__(
            function_name=function_name,
            entity_id=entity_id,
            domo_instance=domo_instance,
            status=status,
            message=message,
        )

In [ ]:
# | export
@patch_to(DomoAccount, cls_method=True)
def generate_create_body(cls, account_name, config):
    return {
        "displayName": account_name,
        "dataProviderType": config.data_provider_type,
        "name": config.data_provider_type,
        "configurations": config.to_json(),
    }


@patch_to(DomoAccount, cls_method=True)
async def create_account(
    cls: DomoAccount,
    account_name: str,
    config: DomoAccount_Config,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    body = cls.generate_create_body(account_name=account_name, config=config)

    res = await account_routes.create_account(
        auth=auth, config_body=body, debug_api=debug_api, session=session
    )

    if not res.is_success:
        raise DomoAccount_CreateAccount_Error(
            entity_id=account_name,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    return await cls.get_from_id(auth=auth, account_id=res.response.get("id"))

In [ ]:
# | export


class DomoAccount_DeleteAccount_Error(de.DomoError):
    def __init__(
        self,
        entity_id,
        domo_instance,
        status,
        message,
        function_name="delete_account",
    ):

        super().__init__(
            function_name=function_name,
            entity_id=entity_id,
            domo_instance=domo_instance,
            status=status,
            message=message,
        )


@patch_to(DomoAccount)
async def delete_account(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    auth = auth or self.auth

    res = await account_routes.delete_account(
        auth=auth, account_id=self.id, debug_api=debug_api, session=session
    )

    if not res.is_success:

        raise DomoAccount_DeleteAccount_Error(
            entity_id=self.id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    return True

In [ ]:
# | export
class ShareAccount_AccessLevel(Enum):
    CAN_VIEW = "CAN_VIEW"
    CAN_EDIT = "CAN_EDIT"
    CAN_SHARE = "CAN_SHARE"


@patch_to(DomoAccount)
async def share_account(
    self,
    auth: dmda.DomoAuth,
    user_id: int,
    is_v2: bool = False,
    access_level: ShareAccount_AccessLevel = ShareAccount_AccessLevel.CAN_VIEW,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    if is_v2:
        share_payload = account_routes.generate_share_account_payload_v2(
            user_id=user_id, access_level=access_level.value
        )

        return await account_routes.share_account_v2(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
        )

    share_payload = account_routes.generate_share_account_payload_v1(
        user_id=user_id, access_level=access_level.value
    )

    return await account_routes.share_account_v1(
        auth=auth,
        account_id=self.id,
        share_payload=share_payload,
        debug_api=debug_api,
        session=session,
    )


In [ ]:
# | export


@dataclass
class DomoAccounts:
    auth: dmda.DomoAuth

In [ ]:
# | export
@patch_to(DomoAccounts, cls_method=True)
async def get_accounts(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    account_name: str = None,
    account_type: AccountConfig = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):

    res = await account_routes.get_accounts(
        auth=auth, debug_api=debug_api, session=session
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    domo_account_ls = [
        DomoAccount._from_json(account_obj, auth=auth) for account_obj in res.response
    ]

    if not account_name and not account_type:
        return domo_account_ls

    filtered_account_ls = domo_account_ls
    if account_name:
        filtered_account_ls = [
            domo_account
            for domo_account in filtered_account_ls
            if domo_account.name.lower() == account_name.lower()
        ]

    if account_type:
        filtered_account_ls = [
            domo_account
            for domo_account in filtered_account_ls
            if domo_account.data_provider_type == account_type.value.data_provider_type
        ]

    return filtered_account_ls

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoAccounts.get_accounts(
    auth=token_auth,
    account_name="DataSet Copy Account",
    account_type=AccountConfig.dataset_copy,
)

[DomoAccount(name='DataSet Copy Account', data_provider_type='dataset-copy', id=1, created_dt=datetime.datetime(2020, 5, 10, 8, 41, 27), modified_dt=datetime.datetime(2020, 5, 10, 8, 41, 27), config=None)]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
